# Neural networks code and tests notebook

# This notebook takes the flux and photon index data and try to predict classes from this data on a :

# - Conv1D network

In [1]:
import numpy as np
from numpy import random
import pandas as pd
import re
import os
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import math
from scipy import interpolate
import sys 
from re import search
from astropy.io import fits
from astropy import units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude
import shutil
from keras.callbacks import CSVLogger
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import glob
from datetime import datetime
from keras.layers import Conv1D, Dense, Dropout, Input, Concatenate, GlobalMaxPooling1D, BatchNormalization,GlobalAveragePooling1D
from keras.layers.convolutional import MaxPooling1D
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
import tensorflow.keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten,LSTM
from keras.utils import to_categorical
from keras.layers import Dense, Activation, Concatenate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit # or StratifiedShuffleSplit
from pandas import DataFrame


In [2]:
pathON="C:/Users/pierr/Documents_kanop/Shps/Pierre_points_for_louis/Bureau/interpo_ONOFF/ON_data/"
pathOFF="C:/Users/pierr/Documents_kanop/Shps/Pierre_points_for_louis/Bureau/interpo_ONOFF/OFF_data/"

In [3]:
#Storing in a list the adresses of all the data files in the notebook

data_files_ON=[]
file_names_ON=[]

data_files_OFF=[]
file_names_OFF=[]
data_files_ALL=[]
file_names_ALL=[]

#Storing in a list the adresses of all the data files in the notebook
for filename in os.listdir(pathOFF):
    f = os.path.join(pathOFF,filename)
    if os.path.isfile(f):
        data_files_OFF.append(f)
        file_names_OFF.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)
for filename in os.listdir(pathON):
    f = os.path.join(pathON,filename)
    if os.path.isfile(f):
        data_files_ON.append(f)
        file_names_ON.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)        

In [4]:
#Using data_files with only the 36 binning
Filter=True
binning=37


if Filter==True:
    
    idx_OFF=[]
    idx_ON=[]
    dataON=[]
    dataOFF=[]
    for i in range(len(data_files_OFF)):
        dataframe = pd.read_csv(data_files_OFF[i])
        lg = len(dataframe)
        
        if lg==binning:
            idx_OFF.append(i)
    
    for i in range(len(data_files_ON)):
        dataframe = pd.read_csv(data_files_ON[i])
        lg = len(dataframe)
#         print(lg)
        if lg==binning:
            idx_ON.append(i)

    #Future input shape : 3511 sources, 2 features (as time series) , x binnings
    for i in range(len(idx_OFF)):

        a=idx_OFF[i]
        dataOFF.append(data_files_OFF[a])
    for i in range(len(idx_ON)):

        a=idx_ON[i]
        dataON.append(data_files_ON[a])

idx = idx_OFF+idx_ON


In [5]:
#3D shape of data, multivariate time series

lgON=len(dataON)
lgOFF=len(dataOFF)
lg=lgON+lgOFF
Labels = []
Class_OFF=[]

nbfeatures=3

a=binning
b=nbfeatures
c=lg

# multivariate
y=np.zeros((2,lg))
bigdata= np.zeros((b,c,a))


for j in range(len(dataOFF)):

    dataframe=pd.read_csv(dataOFF[j],index_col=[0])
    dataframe.columns=['MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']

    bigdata[0][j]=dataframe['Flux']
    bigdata[1][j]=dataframe['Photon Index']*dataframe['Flux']
    bigdata[2][j]=dataframe['Photon Index']

for j in range(len(dataON)):

    v=j+len(dataOFF)
    dataframe=pd.read_csv(dataON[j],index_col=[0])
    dataframe.columns=['MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']

    bigdata[0][v]=dataframe['Flux']
    bigdata[1][v]=dataframe['Photon Index']*dataframe['Flux']
    bigdata[2][v]=dataframe['Photon Index']



for i in range(lgOFF):
    
    Labels.append(int(0))
    
for i in range(lgON):
    
    Labels.append(int(1))


In [6]:
#Reshaping data matrix
bigdata=bigdata.reshape(c,nbfeatures,binning)

In [7]:
from keras.layers import Conv1D
from sklearn.metrics import f1_score

ON_accuracy=[]
OFF_accuracy=[]
fscore=[]
FPOS=[]
FNEG=[]


#Measuring the meaned accuracy of  correct prediction of labellisation for each class  over a defined number of steps
for i in range(200,800):
    
    
    nb_filters=20
    
    weight_for_0 = (1 / lgOFF) * (lg / 2.0)
    weight_for_1 = (1 / lgON) * (lg / 2.0)
    callback=tensorflow.keras.callbacks.EarlyStopping(monitor='binary_accuracy',patience=2, verbose=0, mode='auto',baseline=None, restore_best_weights=True)
    class_weight = {0: weight_for_0, 1: weight_for_1}
    bigdata=bigdata.reshape(c,binning,nbfeatures)
    
    #Conv1D Architecture
    model = Sequential()
    model.add(Conv1D(nb_filters, 1, padding="same", activation="tanh",input_shape=(binning,nbfeatures)))
    model.add(Conv1D(15, 1, padding="same", activation="tanh",input_shape=(nb_filters,nbfeatures)))
    model.add(Conv1D(9, 1, padding="same", activation="tanh",input_shape=(15,nbfeatures)))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(9, activation='tanh'))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss=tensorflow.keras.losses.BinaryCrossentropy(reduction='sum'),optimizer='SGD', metrics=['binary_accuracy'])


    x_train, x_test, y_train, y_test = train_test_split(bigdata, Labels, test_size=0.1, random_state=i)
    y_test2=y_test.copy()
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    
    #fitting data
    model.fit(x_train, y_train,epochs=4,batch_size=1,class_weight=class_weight,callbacks=callback)
    
    print('Iteration no.',i)
    #Obtain the accuracy of prediction for each class
    prediction= model.predict(x_test)
    predicted_labels=[]
    
    for i in range(len(prediction)):
        if prediction[i][0]>0.5:
            predicted_labels.append(0)
        else:
            predicted_labels.append(1)
    on_score=0
    on_nbs=0
    off_nbs=0
    off_score=0
    foff=0
    fon=0
    for i in range(len(y_test)):
        if y_test2[i]==1 :
            on_nbs+=1
            if predicted_labels[i]==1:
                on_score+=1
            else : 
                fon+=1
        if y_test2[i]==0 :
            off_nbs+=1
            if predicted_labels[i]==0:
                off_score+=1 
            else:
                foff+=1
    if on_nbs>0:    
        ON_accuracy.append(100*(on_score/on_nbs))
        FPOS.append(100*(fon/on_nbs))
    OFF_accuracy.append(100*(off_score/off_nbs))
    FNEG.append(100*(foff/off_nbs))
    f1= f1_score(y_test2,predicted_labels,average='weighted')
    fscore.append(f1)
print("Accuracy for ON class: ",np.mean(ON_accuracy) ,"%")
print("Accuracy for OFF class: ",np.mean(OFF_accuracy) ,"%")
print("False Positive rate: ",np.mean(FPOS) ,"%")
print("False Negative rate: ",np.mean(FNEG) ,"%")
print("F1 score: ",np.mean(fscore))

Epoch 1/4
980/980 [==============================] - 2s 2ms/step - loss: 0.5261 - binary_accuracy: 0.6270
Epoch 2/4
980/980 [==============================] - 2s 2ms/step - loss: 0.4067 - binary_accuracy: 0.6908
Epoch 3/4
980/980 [==============================] - 2s 2ms/step - loss: 0.3998 - binary_accuracy: 0.6918
Epoch 4/4
980/980 [==============================] - 1s 1ms/step - loss: 0.3951 - binary_accuracy: 0.6898
Iteration no. 200
4/4 [==============================] - 0s 1ms/step
Epoch 1/4
980/980 [==============================] - 2s 2ms/step - loss: 0.4954 - binary_accuracy: 0.6694
Epoch 2/4
980/980 [==============================] - 2s 2ms/step - loss: 0.4132 - binary_accuracy: 0.6857
Epoch 3/4
980/980 [==============================] - 2s 2ms/step - loss: 0.3995 - binary_accuracy: 0.6857
Epoch 4/4
980/980 [==============================] - 2s 2ms/step - loss: 0.4033 - binary_accuracy: 0.6857
Iteration no. 201
4/4 [==============================] - 0s 2ms/step
Epoch 1/4
980/

4/4 [==============================] - 0s 1ms/step
Epoch 1/4
980/980 [==============================] - 2s 1ms/step - loss: 0.4932 - binary_accuracy: 0.6888
Epoch 2/4
980/980 [==============================] - 1s 1ms/step - loss: 0.4194 - binary_accuracy: 0.6837
Epoch 3/4
980/980 [==============================] - 1s 1ms/step - loss: 0.3994 - binary_accuracy: 0.6837
Iteration no. 237
4/4 [==============================] - 0s 2ms/step
Epoch 1/4
980/980 [==============================] - 2s 1ms/step - loss: 0.4891 - binary_accuracy: 0.7434
Epoch 2/4
980/980 [==============================] - 1s 1ms/step - loss: 0.4672 - binary_accuracy: 0.6898
Epoch 3/4
980/980 [==============================] - 1s 1ms/step - loss: 0.4019 - binary_accuracy: 0.6837
Iteration no. 238
4/4 [==============================] - 0s 1ms/step
Epoch 1/4
980/980 [==============================] - 2s 1ms/step - loss: 0.5373 - binary_accuracy: 0.6577
Epoch 2/4
980/980 [==============================] - 1s 1ms/step - lo

980/980 [==============================] - 1s 1ms/step - loss: 0.4108 - binary_accuracy: 0.6857
Epoch 3/4
980/980 [==============================] - 1s 1ms/step - loss: 0.3959 - binary_accuracy: 0.6857
Epoch 4/4
980/980 [==============================] - 1s 1ms/step - loss: 0.3874 - binary_accuracy: 0.6857
Iteration no. 255
4/4 [==============================] - 0s 1ms/step
Epoch 1/4
980/980 [==============================] - 2s 1ms/step - loss: 0.4739 - binary_accuracy: 0.7541
Epoch 2/4
980/980 [==============================] - 1s 1ms/step - loss: 0.3894 - binary_accuracy: 0.6796
Epoch 3/4
980/980 [==============================] - 1s 1ms/step - loss: 0.4063 - binary_accuracy: 0.7204
Iteration no. 256
4/4 [==============================] - 0s 1ms/step
Epoch 1/4
980/980 [==============================] - 2s 1ms/step - loss: 0.4626 - binary_accuracy: 0.7490
Epoch 2/4
980/980 [==============================] - 1s 1ms/step - loss: 0.3990 - binary_accuracy: 0.6827
Epoch 3/4
980/980 [=====

980/980 [==============================] - 1s 1ms/step - loss: 0.4142 - binary_accuracy: 0.6786
Epoch 3/4
980/980 [==============================] - 1s 1ms/step - loss: 0.3985 - binary_accuracy: 0.6786
Iteration no. 274
4/4 [==============================] - 0s 2ms/step
Epoch 1/4
980/980 [==============================] - 2s 1ms/step - loss: 0.4963 - binary_accuracy: 0.6929
Epoch 2/4
980/980 [==============================] - 1s 1ms/step - loss: 0.4010 - binary_accuracy: 0.6867
Epoch 3/4
980/980 [==============================] - 1s 1ms/step - loss: 0.3863 - binary_accuracy: 0.6837
Iteration no. 275
4/4 [==============================] - 0s 1ms/step
Epoch 1/4
980/980 [==============================] - 2s 1ms/step - loss: 0.5713 - binary_accuracy: 0.7327
Epoch 2/4
980/980 [==============================] - 1s 1ms/step - loss: 0.4474 - binary_accuracy: 0.6694
Epoch 3/4
980/980 [==============================] - 1s 1ms/step - loss: 0.4335 - binary_accuracy: 0.6694
Iteration no. 276
4/4 [=

980/980 [==============================] - 1s 1ms/step - loss: 0.4531 - binary_accuracy: 0.6898
Epoch 3/4
980/980 [==============================] - 1s 1ms/step - loss: 0.4239 - binary_accuracy: 0.6786
Iteration no. 293
4/4 [==============================] - 0s 2ms/step
Epoch 1/4
980/980 [==============================] - 2s 1ms/step - loss: 0.4956 - binary_accuracy: 0.7102
Epoch 2/4
980/980 [==============================] - 1s 1ms/step - loss: 0.4200 - binary_accuracy: 0.6837
Epoch 3/4
980/980 [==============================] - 1s 1ms/step - loss: 0.4052 - binary_accuracy: 0.6837
Iteration no. 294
4/4 [==============================] - 0s 2ms/step
Epoch 1/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5036 - binary_accuracy: 0.6653
Epoch 2/4
980/980 [==============================] - 1s 1ms/step - loss: 0.4210 - binary_accuracy: 0.6745
Epoch 3/4
980/980 [==============================] - 1s 1ms/step - loss: 0.4209 - binary_accuracy: 0.6745
Epoch 4/4
980/980 [=====

...

980/980 [==============================] - 1s 1ms/step - loss: 0.4080 - binary_accuracy: 0.6857
Epoch 3/4
980/980 [==============================] - 1s 1ms/step - loss: 0.3913 - binary_accuracy: 0.6908
Epoch 4/4
980/980 [==============================] - 1s 1ms/step - loss: 0.3901 - binary_accuracy: 0.6878
Iteration no. 682
4/4 [==============================] - 0s 2ms/step
Epoch 1/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5103 - binary_accuracy: 0.7184
Epoch 2/4
980/980 [==============================] - 1s 1ms/step - loss: 0.4148 - binary_accuracy: 0.6847
Epoch 3/4
980/980 [==============================] - 1s 1ms/step - loss: 0.3936 - binary_accuracy: 0.6806
Iteration no. 683
4/4 [==============================] - 0s 3ms/step
Epoch 1/4
980/980 [==============================] - 1s 1ms/step - loss: 0.4730 - binary_accuracy: 0.6867
Epoch 2/4
980/980 [==============================] - 1s 1ms/step - loss: 0.4115 - binary_accuracy: 0.6847
Epoch 3/4
980/980 [=====

980/980 [==============================] - 1s 1ms/step - loss: 0.5113 - binary_accuracy: 0.6602
Epoch 2/4
980/980 [==============================] - 1s 1ms/step - loss: 0.4329 - binary_accuracy: 0.6786
Epoch 3/4
980/980 [==============================] - 1s 1ms/step - loss: 0.4176 - binary_accuracy: 0.6786
Epoch 4/4
980/980 [==============================] - 1s 1ms/step - loss: 0.4164 - binary_accuracy: 0.6786
Iteration no. 701
4/4 [==============================] - 0s 2ms/step
Epoch 1/4
980/980 [==============================] - 2s 1ms/step - loss: 0.4764 - binary_accuracy: 0.7214
Epoch 2/4
980/980 [==============================] - 1s 1ms/step - loss: 0.3859 - binary_accuracy: 0.6847
Epoch 3/4
980/980 [==============================] - 1s 1ms/step - loss: 0.3665 - binary_accuracy: 0.6949
Iteration no. 702
4/4 [==============================] - 0s 2ms/step
Epoch 1/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5105 - binary_accuracy: 0.7036
Epoch 2/4
980/980 [=====

980/980 [==============================] - 1s 1ms/step - loss: 0.3971 - binary_accuracy: 0.6878
Epoch 3/4
980/980 [==============================] - 1s 1ms/step - loss: 0.3791 - binary_accuracy: 0.6969
Epoch 4/4
980/980 [==============================] - 1s 1ms/step - loss: 0.3842 - binary_accuracy: 0.6867
Iteration no. 738
4/4 [==============================] - 0s 2ms/step
Epoch 1/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5267 - binary_accuracy: 0.7000
Epoch 2/4
980/980 [==============================] - 1s 1ms/step - loss: 0.4107 - binary_accuracy: 0.6949
Epoch 3/4
980/980 [==============================] - 1s 1ms/step - loss: 0.3970 - binary_accuracy: 0.6929
Iteration no. 739
4/4 [==============================] - 0s 2ms/step
Epoch 1/4
980/980 [==============================] - 1s 1ms/step - loss: 0.4959 - binary_accuracy: 0.6944
Epoch 2/4
980/980 [==============================] - 1s 1ms/step - loss: 0.4121 - binary_accuracy: 0.6867
Epoch 3/4
980/980 [=====

980/980 [==============================] - 1s 1ms/step - loss: 0.4309 - binary_accuracy: 0.7041
Epoch 3/4
980/980 [==============================] - 1s 1ms/step - loss: 0.4192 - binary_accuracy: 0.6755
Epoch 4/4
980/980 [==============================] - 1s 1ms/step - loss: 0.3903 - binary_accuracy: 0.6755
Iteration no. 757
4/4 [==============================] - 0s 2ms/step
Epoch 1/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5062 - binary_accuracy: 0.7102
Epoch 2/4
980/980 [==============================] - 1s 1ms/step - loss: 0.4217 - binary_accuracy: 0.7082
Epoch 3/4
980/980 [==============================] - 1s 1ms/step - loss: 0.4031 - binary_accuracy: 0.6857
Iteration no. 758
4/4 [==============================] - 0s 2ms/step
Epoch 1/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5562 - binary_accuracy: 0.6862
Epoch 2/4
980/980 [==============================] - 1s 1ms/step - loss: 0.4364 - binary_accuracy: 0.6837
Epoch 3/4
980/980 [=====

980/980 [==============================] - 1s 1ms/step - loss: 0.3932 - binary_accuracy: 0.6806
Iteration no. 795
4/4 [==============================] - 0s 2ms/step
Epoch 1/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5402 - binary_accuracy: 0.6449
Epoch 2/4
980/980 [==============================] - 1s 1ms/step - loss: 0.4209 - binary_accuracy: 0.6888
Epoch 3/4
980/980 [==============================] - 1s 1ms/step - loss: 0.4086 - binary_accuracy: 0.6908
Epoch 4/4
980/980 [==============================] - 1s 1ms/step - loss: 0.4023 - binary_accuracy: 0.6867
Iteration no. 796
4/4 [==============================] - 0s 2ms/step
Epoch 1/4
980/980 [==============================] - 2s 1ms/step - loss: 0.5241 - binary_accuracy: 0.6821
Epoch 2/4
980/980 [==============================] - 1s 1ms/step - loss: 0.4287 - binary_accuracy: 0.6765
Epoch 3/4
980/980 [==============================] - 1s 1ms/step - loss: 0.4265 - binary_accuracy: 0.6765
Iteration no. 797
4/4 [=